In [2]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import nltk
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [4]:
dataset = pd.read_csv('imdb.csv')
dataset.head()
dataset = dataset[:1000]

In [5]:
X = [remove_stopwords(sent) for sent in dataset['review']]
X = [simple_preprocess(sent, deacc=True) for sent in X]
len(X)
X_train, X_test, y_train, y_test = train_test_split(X, dataset['sentiment'], test_size=0.33)

In [6]:
model = Word2Vec(X, min_count=1, size=300, window=10)
model.train(X, total_examples=len(X), epochs=30)

(3430275, 3707430)

In [7]:
X_train_v = []
for sent in X_train:
    sent_vector = np.mean([model.wv[word] for word in sent if word in model.wv], axis=0)
    X_train_v.append(sent_vector)

    
X_test_v = []
for sent in X_test:
    sent_vector = np.mean([model.wv[word] for word in sent if word in model.wv], axis=0)
    X_test_v.append(sent_vector)


In [8]:
mlp = MLPClassifier(max_iter=300)
mlp.fit(X_train_v, y_train)

/Users/hilstah/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=300, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [40]:
mlp.score(X_test_v, y_test)

0.7757575757575758